In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='TRUE'

import warnings
# 忽略所有的 featurewarnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
from NGSIM_env.envs.ngsim_env import NGSIMEnv
from stable_baselines3.common.vec_env import SubprocVecEnv
env_case_ids = [8, 9, 88, 99, 888, 999]

envs = SubprocVecEnv(
    [
        lambda: NGSIMEnv(
            scene="us-101", period=0, vehicle_id=case_id, IDM=False
        )
        for case_id in env_case_ids
    ]
)

In [3]:
from stable_baselines3 import PPO

model = PPO.load("./models/PPO_NGSIM")
import torch
# env = torch.load("./envs/env_9", weights_only=False)

env = NGSIMEnv(scene="us-101", period=0, vehicle_id=999, IDM=False)
torch.save(env, "./envs/env_999")

obs = env.reset()
for _ in range(1000):  # 运行1000个时间步
    env.render(mode="rgb_array")
    action, _states = model.predict(obs, deterministic=True)  # 预测动作
    print(f"action: {action}")
    print(f"_states: {_states}")
    obs, rewards, dones, info = env.step(action)  # 执行动作
    print(f"rewards: {rewards}")
    print(f"dones: {dones}")
    print(f"info: {info}")
    if info["crashed"]:
        print("crashed")
        break

Loading Data...
action: [0 9]
_states: None
rewards: -609.927714459362
dones: True
info: {'velocity': 2.4514807286426787, 'crashed': True, 'offroad': False, 'action': [7.314843035659861, 8.004814205270058, 5], 'features': array([3.73385735e+00, 8.17241487e-01, 3.67572733e-01, 6.20013315e-01,
       5.22858405e-02, 3.72007598e-44, 1.00000000e+00, 0.00000000e+00,
       5.62286343e+00]), 'time': 27}
crashed


In [4]:
# from stable_baselines3 import PPO
# # 加载模型
# model = PPO.load("./models/PPO_NGSIM")

# # 运行推理
# obs = envs.reset()
# for _ in range(1000):  # 运行1000个时间步
#     envs.render(mode="rgb_array")
#     action, _states = model.predict(obs, deterministic=True)  # 预测动作
#     print(f"action: {action}")
#     print(f"_states: {_states}")
#     obs, rewards, dones, info = envs.step(action)  # 执行动作
#     print(f"rewards: {rewards}")
#     print(f"dones: {dones}")
#     print(f"info: {info}")

In [5]:
# # use DQN to train the model
# env = NGSIMEnv(scene='us-101',period=0,vehicle_id=1200,IDM=False)
# print("created env")
# model = DQN(
#     "MlpPolicy",
#     env,
#     verbose=1,
#     learning_rate=1e-3,
#     buffer_size=50000,
#     batch_size=64,
# )
# print("Start training")
# model.learn(total_timesteps=10000)

# model.save("dqn_ngsim")

In [6]:
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from gym import Wrapper

# -------------------------------
# 1. 定义判别器网络
# -------------------------------
class Discriminator(nn.Module):
    def __init__(self, obs_dim, act_dim, hidden_sizes=[64, 64]):
        super(Discriminator, self).__init__()
        layers = []
        input_dim = obs_dim + act_dim  # 将状态和动作拼接后输入
        for h in hidden_sizes:
            layers.append(nn.Linear(input_dim, h))
            layers.append(nn.ReLU())
            input_dim = h
        layers.append(nn.Linear(input_dim, 1))
        layers.append(nn.Sigmoid())  # 输出在0~1之间
        self.net = nn.Sequential(*layers)
    
    def forward(self, obs, act):
        # 假设 obs: [batch, obs_dim]，act: [batch, act_dim]
        x = torch.cat([obs, act], dim=1)
        return self.net(x)

# -------------------------------
# 2. 定义环境包装器，用于替换原始奖励为 imitation reward
# -------------------------------
class GAILRewardWrapper(Wrapper):
    def __init__(self, env, discriminator, device='cpu'):
        super(GAILRewardWrapper, self).__init__(env)
        self.discriminator = discriminator
        self.device = device

    def step(self, action):
        obs, _, done, info = self.env.step(action)
        # 将观测和动作转为 tensor（注意对动作的处理，这里假设动作为标量，转换为 float tensor）
        obs_tensor = torch.tensor(np.array(obs), dtype=torch.float32).unsqueeze(0).to(self.device)
        # 将动作转换为 one-hot 编码（离散动作）
        act_dim = self.action_space.n
        action_onehot = np.zeros(act_dim, dtype=np.float32)
        action_onehot[action] = 1.0
        act_tensor = torch.tensor(action_onehot, dtype=torch.float32).unsqueeze(0).to(self.device)
        with torch.no_grad():
            d_val = self.discriminator(obs_tensor, act_tensor)
        # imitation reward计算：通常使用 -log(1 - D(s,a)) 或 log(D(s,a))
        imitation_reward = -torch.log(1 - d_val + 1e-8).item()
        return obs, imitation_reward, done, info

# -------------------------------
# 3. GAIL 算法封装
# -------------------------------
class GAIL:
    def __init__(self, env_id, expert_data, total_timesteps=100000,
                 disc_lr=1e-3, device='cpu'):
        """
        :param env_id: Gym 环境ID
        :param expert_data: 专家演示数据，格式为 list/dict，其中每个样本包含 (obs, action)
        :param total_timesteps: 总训练步数
        :param disc_lr: 判别器学习率
        :param device: 使用的设备
        """
        self.device = device
        self.env_id = env_id
        self.expert_data = expert_data  # 专家数据应为 list，每个元素为 (obs, action)
        # 构建原始环境
        self.env = DummyVecEnv([lambda: gym.make(env_id)])
        # 获取环境观测和动作空间信息
        obs_dim = self.env.observation_space.shape[0]
        act_space = self.env.action_space
        if isinstance(act_space, gym.spaces.Discrete):
            act_dim = act_space.n
        else:
            act_dim = act_space.shape[0]
        self.obs_dim = obs_dim
        self.act_dim = act_dim
        # 构建判别器
        self.discriminator = Discriminator(obs_dim, act_dim).to(self.device)
        self.disc_optimizer = optim.Adam(self.discriminator.parameters(), lr=disc_lr)
        # 构建 PPO 策略（生成器）
        # 注意：这里先用原始环境构建 PPO，后面在训练过程中，我们会用包装器替换奖励
        self.policy = PPO("MlpPolicy", self.env, verbose=1, device=self.device)

    def update_discriminator(self, batch_size=64, disc_iters=10):
        # 随机采样专家数据和生成器数据
        expert_indices = np.random.choice(len(self.expert_data), batch_size)
        expert_batch = [self.expert_data[i] for i in expert_indices]
        expert_obs = np.array([sample[0] for sample in expert_batch], dtype=np.float32)
        expert_act = np.array([sample[1] for sample in expert_batch], dtype=np.int64)
        # 将专家动作转换为 one-hot 编码
        expert_act_oh = np.zeros((batch_size, self.act_dim), dtype=np.float32)
        expert_act_oh[np.arange(batch_size), expert_act] = 1.0

        # 生成器数据：收集一批 trajectories
        gen_obs = []
        gen_act = []
        obs = self.env.reset()
        for _ in range(batch_size):
            action, _ = self.policy.predict(obs, deterministic=True)
            gen_obs.append(obs[0])
            gen_act.append(action[0])
            obs, _, dones, _ = self.env.step(action)
            if dones[0]:
                obs = self.env.reset()
        gen_obs = np.array(gen_obs, dtype=np.float32)
        gen_act = np.array(gen_act, dtype=np.int64)
        gen_act_oh = np.zeros((batch_size, self.act_dim), dtype=np.float32)
        gen_act_oh[np.arange(batch_size), gen_act] = 1.0

        # 转换为 tensor
        expert_obs_t = torch.tensor(expert_obs, dtype=torch.float32).to(self.device)
        expert_act_t = torch.tensor(expert_act_oh, dtype=torch.float32).to(self.device)
        gen_obs_t = torch.tensor(gen_obs, dtype=torch.float32).to(self.device)
        gen_act_t = torch.tensor(gen_act_oh, dtype=torch.float32).to(self.device)

        # 判别器输出
        d_expert = self.discriminator(expert_obs_t, expert_act_t)
        d_gen = self.discriminator(gen_obs_t, gen_act_t)

        # 判别器损失：二分类交叉熵
        loss_expert = -torch.log(d_expert + 1e-8).mean()
        loss_gen = -torch.log(1 - d_gen + 1e-8).mean()
        disc_loss = loss_expert + loss_gen

        self.disc_optimizer.zero_grad()
        disc_loss.backward()
        self.disc_optimizer.step()
        return disc_loss.item()

    def train(self, timesteps_per_iter=1000, disc_iters=10, total_iters=100):
        """
        交替更新判别器和生成器
        """
        for it in range(total_iters):
            # --- 1. 更新判别器 ---
            disc_loss_avg = 0
            for _ in range(disc_iters):
                disc_loss_avg += self.update_discriminator()
            disc_loss_avg /= disc_iters

            # --- 2. 使用包装器更新生成器（策略） ---
            # 这里我们将 PPO 所使用的环境替换为带有 imitation reward 的环境包装器
            wrapped_env = DummyVecEnv([lambda: GAILRewardWrapper(gym.make(self.env_id),
                                                                   self.discriminator,
                                                                   device=self.device)])
            self.policy.set_env(wrapped_env)
            # 更新策略：这里调用 learn()，注意 timesteps_per_iter 可根据需要调整
            self.policy.learn(total_timesteps=timesteps_per_iter)
            
            print(f"Iter {it+1}/{total_iters}: Disc Loss = {disc_loss_avg:.4f}")

# -------------------------------
# 4. 使用示例
# -------------------------------
if __name__ == '__main__':
    # 假设 expert_data 是提前准备好的专家演示数据，
    # 格式为 list，每个元素为 (obs, action)。
    # 这里为了演示，我们随机生成一些数据（实际应用中应加载真实专家数据）
    env_id = "CartPole-v1"
    env = gym.make(env_id)
    expert_data = []
    for _ in range(1000):
        obs, _ = env.reset()
        done = False
        while not done:
            action = env.action_space.sample()
            expert_data.append((obs, action))
            obs, reward, done, _, info = env.step(action)
            # observation, reward, terminated, truncated, info
    # 创建 GAIL 算法实例
    gail_trainer = GAIL(env_id=env_id, expert_data=expert_data,
                        total_timesteps=100000, device='cpu')
    # 开始训练（交替更新判别器和生成器）
    gail_trainer.train(timesteps_per_iter=1000, disc_iters=10, total_iters=50)


d:\anaconda3\envs\py\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Using cpu device


ValueError: setting an array element with a sequence. The requested array would exceed the maximum number of dimension of 1.